# Import des packages

In [2]:
import duckdb
import pandas as pd

# Chemin et données brutes

In [5]:
path_to_read = "data/raw"
path_to_save = "data/processed"

In [3]:
!whoami

guillaume


In [11]:
data_epci = duckdb.read_csv(f"{path_to_read}/perimetre-epci-a-fp.csv", delimiter=';', ignore_errors=True)
data_sau = duckdb.read_csv(f"{path_to_read}/sau.csv")


## Fichier phyto 

### Phyto par commune avec sau

In [12]:
q_sau2020 = """ 
SELECT 
    epci.*,
    sau.sau2020
FROM data_epci AS epci
INNER JOIN data_sau AS sau
    ON epci.insee = sau.codgeo 
"""

df_phyto_sau = duckdb.sql(q_sau2020)
df_phyto_sau

┌─────────┬───────────┬────────────────────────────────────┬──────────────────┬─────────────┬────────────┬───────────────┬───────────────┬─────────┬─────────┬──────────────┬────────────────────────┬───────────┬───────────┬─────────┐
│  dept   │   siren   │           raison_sociale           │ nature_juridique │ mode_financ │ nb_membres │ total_pop_tot │ total_pop_mun │ dep_com │  insee  │ siren_membre │       nom_membre       │ ptot_2025 │ pmun_2025 │ sau2020 │
│ varchar │   int64   │              varchar               │     varchar      │   varchar   │   int64    │    varchar    │    varchar    │ varchar │ varchar │    int64     │        varchar         │  varchar  │  varchar  │  int64  │
├─────────┼───────────┼────────────────────────────────────┼──────────────────┼─────────────┼────────────┼───────────────┼───────────────┼─────────┼─────────┼──────────────┼────────────────────────┼───────────┼───────────┼─────────┤
│ 01      │ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ CC     

In [13]:
q_sau_totale = """  
SELECT 
    dept,
    siren,
    SUM(sau2020) AS total_sau2020
FROM df_phyto_sau
GROUP BY dept, siren
ORDER BY dept, siren
"""

df_sau_totale = duckdb.sql(q_sau_totale)
df_sau_totale

┌─────────┬───────────┬───────────────┐
│  dept   │   siren   │ total_sau2020 │
│ varchar │   int64   │    int128     │
├─────────┼───────────┼───────────────┤
│ 01      │ 200029999 │          4782 │
│ 01      │ 200040350 │         18245 │
│ 01      │ 200042497 │         10175 │
│ 01      │ 200042935 │         13693 │
│ 01      │ 200069193 │         34497 │
│ 01      │ 200070118 │         10606 │
│ 01      │ 200070555 │         12051 │
│ 01      │ 200071371 │         16816 │
│ 01      │ 200071751 │         67702 │
│ 01      │ 240100610 │          6794 │
│ ·       │     ·     │            ·  │
│ ·       │     ·     │            ·  │
│ ·       │     ·     │            ·  │
│ 974     │ 249740077 │          6828 │
│ 974     │ 249740085 │         10590 │
│ 974     │ 249740093 │          9799 │
│ 974     │ 249740101 │          7786 │
│ 974     │ 249740119 │          3770 │
│ 976     │ 200050532 │           187 │
│ 976     │ 200059871 │          1834 │
│ 976     │ 200060457 │          1066 │


In [6]:
data_phyto

┌─────────┬────────────────────────────┬──────────────────────┬──────────────────────────┬───────────────────────────┬────────────┬─────────────────────┬─────────┬─────────────┬─────────┬────────────────────────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────┬─────────┬────────────────────┐
│  annee  │    nom_region_acheteur     │ code_region_acheteur │ nom_departement_acheteur │ code_departement_acheteur │ code_insee │    libelle_insee    │   amm   │ biocontrole │   uab   │           substance            │     cas     │                                     fonction_sdes                                     │ danger  │ quantite_substance │
│ varchar │          varchar           │       varchar        │         varchar          │          varchar          │  varchar   │       varchar       │ varchar │   varchar   │ varchar │            varchar             │   varchar   │                                        varchar           

In [7]:
q_join_epci_phyto = """
SELECT 
    phyto.annee,
    epci.dept,
    epci.siren,
    TRY_CAST(phyto.quantite_substance AS DOUBLE) AS quantite_substance
FROM data_phyto AS phyto
INNER JOIN data_epci AS epci
    ON epci.insee = phyto.code_insee
ORDER BY dept, annee, siren
"""

phyto = duckdb.sql(q_join_epci_phyto)
phyto

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬─────────┬───────────┬────────────────────┐
│  annee  │  dept   │   siren   │ quantite_substance │
│ varchar │ varchar │   int64   │       double       │
├─────────┼─────────┼───────────┼────────────────────┤
│ 2015    │ 01      │ 200029999 │              1.376 │
│ 2015    │ 01      │ 200029999 │              0.021 │
│ 2015    │ 01      │ 200029999 │              2.674 │
│ 2015    │ 01      │ 200029999 │               0.18 │
│ 2015    │ 01      │ 200029999 │               1.29 │
│ 2015    │ 01      │ 200029999 │              0.938 │
│ 2015    │ 01      │ 200029999 │              0.001 │
│ 2015    │ 01      │ 200029999 │              0.602 │
│ 2015    │ 01      │ 200029999 │              0.029 │
│ 2015    │ 01      │ 200029999 │             59.251 │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│ 2015    │ 01      │ 200040350 │              0.134 │
│ 2015    

### Phytorelative par epci

In [11]:
phyto

┌─────────┬─────────┬───────────┬────────────────────┐
│  annee  │  dept   │   siren   │ quantite_substance │
│ varchar │ varchar │   int64   │       double       │
├─────────┼─────────┼───────────┼────────────────────┤
│ 2015    │ 01      │ 200029999 │              0.049 │
│ 2015    │ 01      │ 200029999 │              0.028 │
│ 2015    │ 01      │ 200029999 │              1.804 │
│ 2015    │ 01      │ 200029999 │              0.198 │
│ 2015    │ 01      │ 200029999 │              0.393 │
│ 2015    │ 01      │ 200029999 │              0.002 │
│ 2015    │ 01      │ 200029999 │              0.382 │
│ 2015    │ 01      │ 200029999 │              2.815 │
│ 2015    │ 01      │ 200029999 │              0.064 │
│ 2015    │ 01      │ 200029999 │              0.041 │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│ 2015    │ 01      │ 200040350 │              1.017 │
│ 2015    

In [ ]:
#Requête pour obtenir la quantité totale de substance par département, par année et par EPCI
#Requête lourde, à optimiser plus tard si besoin
q_phyto_per_year = """ 
SELECT 
    annee,
    dept,
    siren,
    SUM(quantite_substance) AS total_quantite_substance
FROM phyto
GROUP BY annee, dept, siren
ORDER BY dept, annee, siren
"""

phyto_per_year = duckdb.sql(q_phyto_per_year)
phyto_per_year

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

IOException: IO Error: Could not write file ".tmp/duckdb_temp_block-4611686018427494457.block": No space left on device

In [8]:
data_phyto_relative = duckdb.read_csv(f"{path_to_save}/phyto_relative_csv.csv")
data_phyto_relative


┌───────────┬─────────┬─────────┬───────────────┐
│   siren   │ pop_tot │ dep_com │ total_relatif │
│   int64   │ varchar │ varchar │    double     │
├───────────┼─────────┼─────────┼───────────────┤
│ 200070852 │ 21 965  │ 01      │          1.06 │
│ 200070118 │ 21 470  │ 01      │          1.81 │
│ 240100750 │ 106 924 │ 01      │          0.89 │
│ 240100883 │ 84 625  │ 01      │          4.32 │
│ 200040590 │ 75 051  │ 01      │          6.09 │
│ 240100800 │ 25 583  │ 01      │           1.0 │
│ 200029999 │ 15 156  │ 01      │          0.96 │
│ 200042497 │ 41 975  │ 01      │          1.94 │
│ 200042935 │ 65 428  │ 01      │          0.21 │
│ 200069193 │ 41 461  │ 01      │          5.48 │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│ 249730045 │ 153 249 │ 973     │          0.07 │
│ 249740077 │ 185 848 │ 974     │          1.29 │
│ 249740101 │ 221 532 │ 974     │          0.32 │


In [17]:
q_phyto_relative = """ 
SELECT
    p2.dept,
    p1.siren,
    TRY_CAST(REPLACE(p1.pop_tot,' ','') AS DOUBLE) AS pop_tot,
    p1.total_relatif AS total_relatif,
    p2.total_sau2020
FROM data_phyto_relative AS p1
INNER JOIN df_sau_totale AS p2 ON p1.siren = p2.siren
ORDER BY p2.dept, p1.siren
"""

df_phyto_relative = duckdb.sql(q_phyto_relative)
df_phyto_relative.to_csv(f"{path_to_save}/phyto_relative_csv.csv")